# Wolf-Sheep Predation Model

For this exercise we refer to the original implementation from Wilensky (1997) in `Mesa`. As reference, we used the code from `Mesa`'s example for this case study: https://github.com/projectmesa/mesa/tree/main/examples/wolf_sheep

Before you start, we reccomend you to go ahead and do the basic `Mesa` tutorial from:
https://mesa.readthedocs.io/en/latest/tutorials/intro_tutorial.html

We will visit this tutorial in class.


## Summary

A simple ecological model, consisting of three agent types: wolves, sheep, and grass. The wolves and the sheep wander around the grid at random. Wolves and sheep both expend energy moving around, and replenish it by eating. Sheep eat grass, and wolves eat sheep if they end up on the same grid cell.

If wolves and sheep have enough energy, they reproduce, creating a new wolf or sheep (in this simplified model, only one parent is needed for reproduction). The grass on each cell regrows at a constant rate. If any wolves and sheep run out of energy, they die. 


In [13]:
import mesa
import numpy as np
import plotly as py
import plotly.graph_objects as go

Both sheeps and wolf roam around in random fashion. Let's first create an instance of an `Mesa` agent class that we'll call `RandomWalker`. Both sheeps and wolfs will be random walkers

In [14]:
"""
Generalized behavior for random walking, one grid cell at a time.
"""

class RandomWalker(mesa.Agent):
    """
    Class implementing random walker methods in a generalized manner.
    Not intended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        moore: If True, may move in all 8 directions.
                Otherwise, only up, down, left, right.
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

Now, lets define our two agents, `Sheep` and `Wolf` as `RandomWalker` agents

In [15]:
class Sheep(RandomWalker):
    """
    A sheep that walks around, reproduces (asexually) and gets eaten.
    The init is the same as the RandomWalker.
    """

    energy = None

    def __init__(self, unique_id, pos, model, moore, energy=None):
        super().__init__(unique_id, pos, model, moore=moore)
        self.energy = energy

    def step(self):
        """
        A model step. Move, then eat grass and reproduce.
        """
        self.random_move()
        living = True

        if self.model.grass:
            # Reduce energy
            self.energy -= 1

            # If there is grass available, eat it
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            grass_patch = [obj for obj in this_cell if isinstance(obj, GrassPatch)][0]
            if grass_patch.fully_grown:
                self.energy += self.model.sheep_gain_from_food
                grass_patch.fully_grown = False

            # Death
            if self.energy < 0:
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)
                living = False

        if living and self.random.random() < self.model.sheep_reproduce:
            # Create a new sheep:
            if self.model.grass:
                self.energy /= 2
            lamb = Sheep(
                self.model.next_id(), self.pos, self.model, self.moore, self.energy
            )
            self.model.grid.place_agent(lamb, self.pos)
            self.model.schedule.add(lamb)

In [16]:
class Wolf(RandomWalker):
    """
    A wolf that walks around, reproduces (asexually) and eats sheep.
    """

    energy = None

    def __init__(self, unique_id, pos, model, moore, energy=None):
        super().__init__(unique_id, pos, model, moore=moore)
        self.energy = energy

    def step(self):
        self.random_move()
        self.energy -= 1

        # If there are sheep present, eat one
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        sheep = [obj for obj in this_cell if isinstance(obj, Sheep)]
        if len(sheep) > 0:
            sheep_to_eat = self.random.choice(sheep)
            self.energy += self.model.wolf_gain_from_food

            # Kill the sheep
            self.model.grid.remove_agent(sheep_to_eat)
            self.model.schedule.remove(sheep_to_eat)

        # Death or reproduction
        if self.energy < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            if self.random.random() < self.model.wolf_reproduce:
                # Create a new wolf cub
                self.energy /= 2
                cub = Wolf(
                    self.model.next_id(), self.pos, self.model, self.moore, self.energy
                )
                self.model.grid.place_agent(cub, cub.pos)
                self.model.schedule.add(cub)

And the grass, `GrassPatch`, as a regular (not random walker) agent...

In [17]:
class GrassPatch(mesa.Agent):
    """
    A patch of grass that grows at a fixed rate and it is eaten by sheep
    """

    def __init__(self, unique_id, pos, model, fully_grown, countdown):
        """
        Creates a new patch of grass
        Args:
            grown: (boolean) Whether the patch of grass is fully grown or not
            countdown: Time for the patch of grass to be fully grown again
        """
        super().__init__(unique_id, model)
        self.fully_grown = fully_grown
        self.countdown = countdown
        self.pos = pos

    def step(self):
        if not self.fully_grown:
            if self.countdown <= 0:
                # Set as fully grown
                self.fully_grown = True
                self.countdown = self.model.grass_regrowth_time
            else:
                self.countdown -= 1

Now let's create our `Mesa` model. Initiate the parameters of interest (self explanatory), create all agents, define the `step` function and the `run_model`

In [23]:
class WolfSheep(mesa.Model):
    """
    Wolf-Sheep Predation Model
    """

    height = 20
    width = 20

    initial_sheep = 100
    initial_wolves = 50

    sheep_reproduce = 0.04
    wolf_reproduce = 0.05

    wolf_gain_from_food = 20

    grass = False
    grass_regrowth_time = 30
    sheep_gain_from_food = 4

    verbose = True  # Print-monitoring

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(
        self,
        width=20,
        height=20,
        initial_sheep=100,
        initial_wolves=50,
        wolf_reproduce=0.05,
        wolf_gain_from_food=20,
        grass=False,
        grass_regrowth_time=30,
        sheep_gain_from_food=4,
    ):
        """
        Create a new Wolf-Sheep model with the given parameters.
        Args:
            initial_sheep: Number of sheep to start with
            initial_wolves: Number of wolves to start with
            sheep_reproduce: Probability of each sheep reproducing each step
            wolf_reproduce: Probability of each wolf reproducing each step
            wolf_gain_from_food: Energy a wolf gains from eating a sheep
            grass: Whether to have the sheep eat grass for energy
            grass_regrowth_time: How long it takes for a grass patch to regrow
                                 once it is eaten
            sheep_gain_from_food: Energy sheep gain from grass, if enabled.
        """
        super().__init__()
        # Set parameters
        self.width = width
        self.height = height
        self.initial_sheep = initial_sheep
        self.initial_wolves = initial_wolves
        self.sheep_reproduce = sheep_reproduce
        self.wolf_reproduce = wolf_reproduce
        self.wolf_gain_from_food = wolf_gain_from_food
        self.grass = grass
        self.grass_regrowth_time = grass_regrowth_time
        self.sheep_gain_from_food = sheep_gain_from_food
        
        self.grass_count = 0
        self.wolves_evolution = []
        self.sheep_evolution = []
        self.grass_evolution = []

        self.schedule = RandomActivationByTypeFiltered(self)
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)
        self.datacollector = mesa.DataCollector(
            {
                "Wolves": lambda m: m.schedule.get_type_count(Wolf),
                "Sheep": lambda m: m.schedule.get_type_count(Sheep),
                "Grass": lambda m: m.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
            }
        )

        # Create sheep:
        for i in range(self.initial_sheep):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.sheep_gain_from_food)
            sheep = Sheep(self.next_id(), (x, y), self, True, energy)
            self.grid.place_agent(sheep, (x, y))
            self.schedule.add(sheep)

        # Create wolves
        for i in range(self.initial_wolves):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.wolf_gain_from_food)
            wolf = Wolf(self.next_id(), (x, y), self, True, energy)
            self.grid.place_agent(wolf, (x, y))
            self.schedule.add(wolf)

        # Create grass patches
        if self.grass:
            for agent, x, y in self.grid.coord_iter():

                fully_grown = self.random.choice([True, False])

                if fully_grown:
                    countdown = self.grass_regrowth_time
                else:
                    countdown = self.random.randrange(self.grass_regrowth_time)

                patch = GrassPatch(self.next_id(), (x, y), self, fully_grown, countdown)
                self.grid.place_agent(patch, (x, y))
                self.schedule.add(patch)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print(
                [
                    self.schedule.time,
                    self.schedule.get_type_count(Wolf),
                    self.schedule.get_type_count(Sheep),
                    self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
                    #self.grass_count,
                ]
            )

    def run_model(self, step_count=100):

        if self.verbose:
            print("Initial number wolves: ", self.schedule.get_type_count(Wolf))
            print("Initial number sheep: ", self.schedule.get_type_count(Sheep))
            print(
                "Initial number grass: ",
                self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
                #self.grass_count,
            )

        for i in range(step_count):
            self.step()
            self.wolves_evolution.append(self.schedule.get_type_count(Wolf))
            self.sheep_evolution.append(self.schedule.get_type_count(Sheep))
            self.grass_evolution.append(self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown))
            
        if self.verbose:
            print("")
            print("Final number wolves: ", self.schedule.get_type_count(Wolf))
            print("Final number sheep: ", self.schedule.get_type_count(Sheep))
            print(
                "Final number grass: ",
                self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
                #self.grass_count,
            )

Small extension to `Mesa`: Since we want are interest in computing the fully grown grass, we need a scheduler that has a `get_type_count` method to allow for filtering of agents by a function before counting.

In [24]:
from typing import Type, Callable

class RandomActivationByTypeFiltered(mesa.time.RandomActivationByType):
    """
    A scheduler that overrides the get_type_count method to allow for filtering
    of agents by a function before counting.
    Example:
    >>> scheduler = RandomActivationByTypeFiltered(model)
    >>> scheduler.get_type_count(AgentA, lambda agent: agent.some_attribute > 10)
    """

    def get_type_count(
        self,
        type_class: Type[mesa.Agent],
        filter_func: Callable[[mesa.Agent], bool] = None,
    ) -> int:
        """
        Returns the current number of agents of certain type in the queue that satisfy the filter function.
        """
        count = 0
        for agent in self.agents_by_type[type_class].values():
            if filter_func is None or filter_func(agent):
                count += 1
        return count

Now, let's define our model initial parameters. We will keep the default onws for now, but turn on the `Grass` agent.

In [29]:
model = WolfSheep(grass=True)

And let's run it.

In [30]:
model.run_model()

Initial number wolves:  50
Initial number sheep:  100
Initial number grass:  185
[1, 53, 84, 147]
[2, 55, 75, 137]
[3, 57, 65, 119]
[4, 59, 59, 107]
[5, 62, 45, 108]
[6, 63, 41, 109]
[7, 69, 36, 112]
[8, 72, 27, 112]
[9, 76, 25, 115]
[10, 76, 17, 127]
[11, 70, 19, 133]
[12, 69, 16, 137]
[13, 68, 12, 146]
[14, 64, 11, 154]
[15, 64, 11, 157]
[16, 66, 9, 160]
[17, 66, 8, 166]
[18, 62, 7, 166]
[19, 59, 8, 171]
[20, 58, 5, 177]
[21, 55, 4, 180]
[22, 56, 4, 188]
[23, 53, 4, 192]
[24, 48, 5, 198]
[25, 51, 5, 203]
[26, 47, 5, 210]
[27, 44, 3, 213]
[28, 43, 2, 219]
[29, 44, 2, 224]
[30, 40, 2, 233]
[31, 37, 1, 273]
[32, 36, 1, 291]
[33, 37, 1, 309]
[34, 35, 1, 308]
[35, 37, 1, 326]
[36, 38, 2, 326]
[37, 32, 2, 338]
[38, 33, 2, 336]
[39, 31, 2, 340]
[40, 29, 2, 347]
[41, 29, 2, 346]
[42, 26, 2, 352]
[43, 24, 2, 352]
[44, 20, 2, 356]
[45, 18, 2, 363]
[46, 16, 1, 368]
[47, 14, 1, 371]
[48, 12, 1, 371]
[49, 13, 1, 377]
[50, 9, 1, 378]
[51, 8, 1, 378]
[52, 9, 1, 379]
[53, 8, 1, 380]
[54, 6, 1, 380]


Now, let's plot the evolution of our populations:

In [31]:
steps = [i for i in range(1, 100)]

fig = go.Figure([
    
    go.Scatter(
        name='Sheeps',
        x=steps,
        y=model.sheep_evolution,
        mode='lines',
        line=dict(width=1),
        showlegend=True
    ),
    go.Scatter(
        name='Wolves',
        x=steps,
        y=model.wolves_evolution,
        mode='lines',
        line=dict(width=1),
        showlegend=True
    ),
    go.Scatter(
        name='Grass',
        x=steps,
        y=model.grass_evolution,
        marker=dict(color="#444"),
        line=dict(width=1),
        mode='lines',
        showlegend=True
    )
])

fig.show()